# Bài 1: Hãy cài đặt game Cờ caro với giải thuật Reinforcement Learning

In [44]:
import numpy as np
import random

# Kích thước bàn cờ
BOARD_SIZE = 3

# Hằng số
EMPTY = 0
PLAYER_X = 1
PLAYER_O = 2
DRAW = 3

# Tạo bàn cờ trống
def create_board():
    return np.zeros((BOARD_SIZE, BOARD_SIZE))

# Kiểm tra trạng thái kết thúc
def is_game_over(board):
    # Kiểm tra hàng ngang
    for i in range(BOARD_SIZE):
        if board[i][0] == board[i][1] == board[i][2] != EMPTY:
            return True, board[i][0]

    # Kiểm tra hàng dọc
    for i in range(BOARD_SIZE):
        if board[0][i] == board[1][i] == board[2][i] != EMPTY:
            return True, board[0][i]

    # Kiểm tra đường chéo chính
    if board[0][0] == board[1][1] == board[2][2] != EMPTY:
        return True, board[0][0]

    # Kiểm tra đường chéo phụ
    if board[0][2] == board[1][1] == board[2][0] != EMPTY:
        return True, board[0][2]

    # Kiểm tra hòa
    if np.all(board != EMPTY):
        return True, DRAW

    return False, None

# Tìm các nước đi hợp lệ
def get_valid_moves(board):
    return np.argwhere(board == EMPTY)

# Chọn nước đi ngẫu nhiên
def choose_random_move(valid_moves):
    return random.choice(valid_moves)

# Thực hiện nước đi
def make_move(board, move, player):
    board[move[0]][move[1]] = player

# Hàm Q-learning
def q_learning(board, num_episodes, learning_rate, discount_factor, exploration_rate):
    Q = np.zeros((BOARD_SIZE, BOARD_SIZE, 2))
    for episode in range(num_episodes):
        current_board = create_board()
        while True:
            valid_moves = get_valid_moves(current_board)
            if np.random.rand() < exploration_rate:
                move = choose_random_move(valid_moves)
            else:
                move_values = [Q[move[0]][move[1]][PLAYER_X - 1] for move in valid_moves]
                max_value = max(move_values)
                best_moves = [move for move, value in zip(valid_moves, move_values) if value == max_value]
                move = random.choice(best_moves)

            make_move(current_board, move, PLAYER_X)
            game_over, winner = is_game_over(current_board)
            if game_over:
                if winner == PLAYER_X:
                    reward = 1
                elif winner == PLAYER_O:
                    reward = -1
                else:
                    reward = 0
                Q[move[0]][move[1]][PLAYER_X - 1] += learning_rate * (reward - Q[move[0]][move[1]][PLAYER_X - 1])
                break
            else:
                next_moves = get_valid_moves(current_board)
                next_move = random.choice(next_moves)
                make_move(current_board, next_move, PLAYER_O)
                reward = 0
                Q[move[0]][move[1]][PLAYER_X - 1] += learning_rate * (reward + discount_factor * np.max(Q[next_move[0]][next_move[1]]) - Q[move[0]][move[1]][PLAYER_X - 1])
    return Q

# Hàm chơi với người
def play_with_human(Q):
    board = create_board()
    while True:
        print(board)
        row = int(input("Nhập số hàng: "))
        col = int(input("Nhập số cột: "))
        move = (row, col)
        make_move(board, move, PLAYER_X)
        game_over, winner = is_game_over(board)
        if game_over:
            print(board)
            if winner == PLAYER_X:
                print("Bạn đã thắng!")
            elif winner == PLAYER_O:
                print("Bạn đã thua!")
            else:
                print("Hòa!")
            break
        else:
            valid_moves = get_valid_moves(board)
            best_move = valid_moves[np.argmax(Q[valid_moves[:, 0], valid_moves[:, 1], PLAYER_X - 1])]
            make_move(board, best_move, PLAYER_O)
            game_over, winner = is_game_over(board)
            if game_over:
                print(board)
                if winner == PLAYER_X:
                    print("Bạn đã thắng!")
                elif winner == PLAYER_O:
                    print("Bạn đã thua!")
                else:
                    print("Hòa!")
                break

# Thực hiện Q-learning và chơi với người
Q = q_learning(create_board(), num_episodes=1000, learning_rate=0.1, discount_factor=0.9, exploration_rate=0.1)
play_with_human(Q)


[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
Nhập số hàng: 0
Nhập số cột: 1
[[0. 1. 0.]
 [0. 2. 0.]
 [0. 0. 0.]]
Nhập số hàng: 1 
Nhập số cột: 0
[[0. 1. 2.]
 [1. 2. 0.]
 [0. 0. 0.]]
Nhập số hàng: 0 
Nhập số cột: 0
[[1. 1. 2.]
 [1. 2. 0.]
 [2. 0. 0.]]
Bạn đã thua!


## Bài 2: Cải tiến bài 1 với giải thuật Deep Reinforcement Learning